In [3]:
import pandas as pd
import numpy as np



In [6]:
merged_df=pd.read_csv("./data/merge.csv")

In [7]:
# ----------------------
# 1. 重新生成MA10特征
# ----------------------
def add_technical_features(df):
    # 移动平均线
    df['MA5'] = df.groupby('symbol')['close'].transform(lambda x: x.rolling(5).mean())
    df['MA10'] = df.groupby('symbol')['close'].transform(lambda x: x.rolling(10).mean())  # 修复点
    df['MA20'] = df.groupby('symbol')['close'].transform(lambda x: x.rolling(20).mean())
    
    # RSI（14分钟窗口）
    delta = df.groupby('symbol')['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.groupby(df['symbol']).transform(lambda x: x.rolling(14).mean())
    avg_loss = loss.groupby(df['symbol']).transform(lambda x: x.rolling(14).mean())
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema12 = df.groupby('symbol')['close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
    ema26 = df.groupby('symbol')['close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
    df['MACD'] = ema12 - ema26
    df['Signal'] = df.groupby('symbol')['MACD'].transform(lambda x: x.ewm(span=9, adjust=False).mean())
    
    # 其他特征...
    return df

# 重新生成特征
merged_df = add_technical_features(merged_df)


In [8]:
# ----------------------
# 2. 检查数据列
# ----------------------
required_features = ['MA5', 'MA10', 'MA20', 'RSI', 'MACD', 'Signal']
missing = [col for col in required_features if col not in merged_df.columns]
if missing:
    raise ValueError(f"缺失关键特征列: {missing}")
else:
    print("所有特征列已正确生成！")

# ----------------------
# 3. 重新生成序列数据
# ----------------------
def create_sequences(data, window=60):
    X, y = [], []
    for i in range(len(data) - window - 30):  # 预留30分钟计算涨跌幅
        X.append(data[i:i+window])
        y.append(data[i+window+30]["涨跌幅"])  # 未来第30分钟的标签
    return np.array(X), np.array(y)

# 按品种生成序列数据
symbol_data = merged_df[merged_df["symbol"] == "HC"]  # 以热轧卷板为例
X_seq, y_seq = create_sequences(symbol_data[required_features + ['close', 'volume']].values, window=60)

所有特征列已正确生成！


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices